# Frontend Testing Overview

A comprehensive guide to testing strategies, tools, and patterns for modern frontend applications.

## Testing Pyramid & Types

| Type | Scope | Speed | Cost | Tools |
|------|-------|-------|------|-------|
| **Unit** | Single function/component | Fast | Low | Jest, Vitest |
| **Integration** | Multiple components together | Medium | Medium | React Testing Library |
| **E2E** | Full user workflows | Slow | High | Cypress, Playwright |

```
        /\
       /E2E\        <- Few, critical paths
      /------\
     /Integr. \     <- Key interactions
    /----------\
   /   Unit     \   <- Many, fast tests
  /--------------\
```

## Jest Basics

Jest is the most popular JavaScript testing framework with built-in mocking, assertions, and coverage.

```javascript
// jest.config.js
module.exports = {
  testEnvironment: 'jsdom',
  setupFilesAfterEnv: ['<rootDir>/jest.setup.js'],
  moduleNameMapper: {
    '^@/(.*)$': '<rootDir>/src/$1',
    '\\.(css|scss)$': 'identity-obj-proxy'
  },
  collectCoverageFrom: ['src/**/*.{js,jsx,ts,tsx}']
};
```

### Core Testing Patterns

```javascript
// utils.test.js
import { formatPrice, validateEmail, debounce } from './utils';

describe('formatPrice', () => {
  it('formats number to currency string', () => {
    expect(formatPrice(1234.56)).toBe('$1,234.56');
  });

  it('handles zero', () => {
    expect(formatPrice(0)).toBe('$0.00');
  });

  it.each([
    [100, '$100.00'],
    [1000, '$1,000.00'],
    [1000000, '$1,000,000.00']
  ])('formats %i as %s', (input, expected) => {
    expect(formatPrice(input)).toBe(expected);
  });
});

describe('validateEmail', () => {
  it('returns true for valid emails', () => {
    expect(validateEmail('user@example.com')).toBe(true);
  });

  it('returns false for invalid emails', () => {
    expect(validateEmail('invalid')).toBe(false);
    expect(validateEmail('@no-local.com')).toBe(false);
  });
});

describe('debounce', () => {
  beforeEach(() => jest.useFakeTimers());
  afterEach(() => jest.useRealTimers());

  it('delays function execution', () => {
    const fn = jest.fn();
    const debounced = debounce(fn, 300);

    debounced();
    expect(fn).not.toHaveBeenCalled();

    jest.advanceTimersByTime(300);
    expect(fn).toHaveBeenCalledTimes(1);
  });
});
```

## React Testing Library Patterns

RTL encourages testing components the way users interact with them.

### Query Priority
1. **Accessible queries**: `getByRole`, `getByLabelText`, `getByPlaceholderText`
2. **Semantic queries**: `getByText`, `getByAltText`, `getByTitle`
3. **Test IDs**: `getByTestId` (last resort)

```javascript
// LoginForm.test.jsx
import { render, screen, waitFor } from '@testing-library/react';
import userEvent from '@testing-library/user-event';
import LoginForm from './LoginForm';

describe('LoginForm', () => {
  const mockOnSubmit = jest.fn();
  const user = userEvent.setup();

  beforeEach(() => {
    mockOnSubmit.mockClear();
  });

  it('renders all form elements', () => {
    render(<LoginForm onSubmit={mockOnSubmit} />);

    expect(screen.getByRole('textbox', { name: /email/i })).toBeInTheDocument();
    expect(screen.getByLabelText(/password/i)).toBeInTheDocument();
    expect(screen.getByRole('button', { name: /sign in/i })).toBeInTheDocument();
  });

  it('submits form with valid credentials', async () => {
    render(<LoginForm onSubmit={mockOnSubmit} />);

    await user.type(screen.getByRole('textbox', { name: /email/i }), 'user@test.com');
    await user.type(screen.getByLabelText(/password/i), 'password123');
    await user.click(screen.getByRole('button', { name: /sign in/i }));

    expect(mockOnSubmit).toHaveBeenCalledWith({
      email: 'user@test.com',
      password: 'password123'
    });
  });

  it('shows validation errors for empty fields', async () => {
    render(<LoginForm onSubmit={mockOnSubmit} />);

    await user.click(screen.getByRole('button', { name: /sign in/i }));

    expect(await screen.findByText(/email is required/i)).toBeInTheDocument();
    expect(screen.getByText(/password is required/i)).toBeInTheDocument();
    expect(mockOnSubmit).not.toHaveBeenCalled();
  });

  it('disables submit button while loading', async () => {
    render(<LoginForm onSubmit={mockOnSubmit} isLoading />);

    expect(screen.getByRole('button', { name: /signing in/i })).toBeDisabled();
  });
});
```

## Component Testing Strategies

### Testing Custom Hooks

```javascript
// useCounter.test.js
import { renderHook, act } from '@testing-library/react';
import useCounter from './useCounter';

describe('useCounter', () => {
  it('initializes with default value', () => {
    const { result } = renderHook(() => useCounter());
    expect(result.current.count).toBe(0);
  });

  it('initializes with custom value', () => {
    const { result } = renderHook(() => useCounter(10));
    expect(result.current.count).toBe(10);
  });

  it('increments count', () => {
    const { result } = renderHook(() => useCounter());
    act(() => result.current.increment());
    expect(result.current.count).toBe(1);
  });
});
```

### Testing with Context/Providers

```javascript
// test-utils.jsx
import { render } from '@testing-library/react';
import { ThemeProvider } from './ThemeContext';
import { AuthProvider } from './AuthContext';
import { QueryClient, QueryClientProvider } from '@tanstack/react-query';

const createTestQueryClient = () => new QueryClient({
  defaultOptions: {
    queries: { retry: false },
    mutations: { retry: false }
  }
});

export function renderWithProviders(ui, { theme = 'light', user = null, ...options } = {}) {
  const queryClient = createTestQueryClient();

  function Wrapper({ children }) {
    return (
      <QueryClientProvider client={queryClient}>
        <AuthProvider initialUser={user}>
          <ThemeProvider initialTheme={theme}>
            {children}
          </ThemeProvider>
        </AuthProvider>
      </QueryClientProvider>
    );
  }

  return {
    ...render(ui, { wrapper: Wrapper, ...options }),
    queryClient
  };
}

// UserProfile.test.jsx
import { renderWithProviders } from './test-utils';
import UserProfile from './UserProfile';

it('shows user name when authenticated', () => {
  renderWithProviders(<UserProfile />, {
    user: { name: 'John Doe', email: 'john@test.com' }
  });

  expect(screen.getByText('John Doe')).toBeInTheDocument();
});
```

## Mocking APIs

### MSW (Mock Service Worker) - Recommended Approach

```javascript
// mocks/handlers.js
import { rest } from 'msw';

export const handlers = [
  rest.get('/api/users/:id', (req, res, ctx) => {
    const { id } = req.params;
    return res(
      ctx.status(200),
      ctx.json({ id, name: 'John Doe', email: 'john@test.com' })
    );
  }),

  rest.post('/api/login', async (req, res, ctx) => {
    const { email, password } = await req.json();
    if (email === 'user@test.com' && password === 'correct') {
      return res(ctx.json({ token: 'fake-jwt-token' }));
    }
    return res(ctx.status(401), ctx.json({ error: 'Invalid credentials' }));
  }),

  rest.get('/api/products', (req, res, ctx) => {
    const page = req.url.searchParams.get('page') || '1';
    return res(ctx.json({
      products: [{ id: 1, name: 'Product 1' }],
      page: parseInt(page),
      totalPages: 5
    }));
  })
];

// mocks/server.js
import { setupServer } from 'msw/node';
import { handlers } from './handlers';
export const server = setupServer(...handlers);

// jest.setup.js
import { server } from './mocks/server';
beforeAll(() => server.listen());
afterEach(() => server.resetHandlers());
afterAll(() => server.close());
```

### Testing with MSW

```javascript
// UserDashboard.test.jsx
import { rest } from 'msw';
import { server } from './mocks/server';
import { renderWithProviders } from './test-utils';
import UserDashboard from './UserDashboard';

describe('UserDashboard', () => {
  it('displays user data on successful fetch', async () => {
    renderWithProviders(<UserDashboard userId="123" />);

    expect(screen.getByText(/loading/i)).toBeInTheDocument();
    expect(await screen.findByText('John Doe')).toBeInTheDocument();
  });

  it('displays error message on API failure', async () => {
    server.use(
      rest.get('/api/users/:id', (req, res, ctx) =>
        res(ctx.status(500), ctx.json({ error: 'Server error' }))
      )
    );

    renderWithProviders(<UserDashboard userId="123" />);

    expect(await screen.findByText(/error loading user/i)).toBeInTheDocument();
  });

  it('handles network errors gracefully', async () => {
    server.use(
      rest.get('/api/users/:id', (req, res) => res.networkError('Failed to connect'))
    );

    renderWithProviders(<UserDashboard userId="123" />);

    expect(await screen.findByText(/connection failed/i)).toBeInTheDocument();
  });
});
```

## E2E Testing with Playwright

Playwright provides cross-browser testing with excellent developer experience.

```javascript
// playwright.config.ts
import { defineConfig, devices } from '@playwright/test';

export default defineConfig({
  testDir: './e2e',
  fullyParallel: true,
  retries: process.env.CI ? 2 : 0,
  workers: process.env.CI ? 1 : undefined,
  reporter: 'html',
  use: {
    baseURL: 'http://localhost:3000',
    trace: 'on-first-retry',
    screenshot: 'only-on-failure'
  },
  projects: [
    { name: 'chromium', use: { ...devices['Desktop Chrome'] } },
    { name: 'firefox', use: { ...devices['Desktop Firefox'] } },
    { name: 'mobile', use: { ...devices['iPhone 13'] } }
  ],
  webServer: {
    command: 'npm run dev',
    url: 'http://localhost:3000',
    reuseExistingServer: !process.env.CI
  }
});
```

### E2E Test Examples

```javascript
// e2e/auth.spec.ts
import { test, expect } from '@playwright/test';

test.describe('Authentication', () => {
  test('user can sign in and access dashboard', async ({ page }) => {
    await page.goto('/login');

    await page.getByLabel('Email').fill('user@example.com');
    await page.getByLabel('Password').fill('password123');
    await page.getByRole('button', { name: 'Sign In' }).click();

    await expect(page).toHaveURL('/dashboard');
    await expect(page.getByRole('heading', { name: 'Welcome back' })).toBeVisible();
  });

  test('shows error for invalid credentials', async ({ page }) => {
    await page.goto('/login');

    await page.getByLabel('Email').fill('wrong@example.com');
    await page.getByLabel('Password').fill('wrongpassword');
    await page.getByRole('button', { name: 'Sign In' }).click();

    await expect(page.getByText('Invalid email or password')).toBeVisible();
    await expect(page).toHaveURL('/login');
  });
});

// e2e/checkout.spec.ts
test.describe('Checkout Flow', () => {
  test.beforeEach(async ({ page }) => {
    // Authenticate before each test
    await page.goto('/login');
    await page.getByLabel('Email').fill('user@example.com');
    await page.getByLabel('Password').fill('password123');
    await page.getByRole('button', { name: 'Sign In' }).click();
    await page.waitForURL('/dashboard');
  });

  test('complete purchase flow', async ({ page }) => {
    // Add item to cart
    await page.goto('/products');
    await page.getByRole('button', { name: 'Add to Cart' }).first().click();
    await expect(page.getByTestId('cart-count')).toHaveText('1');

    // Proceed to checkout
    await page.goto('/cart');
    await page.getByRole('button', { name: 'Checkout' }).click();

    // Fill shipping details
    await page.getByLabel('Address').fill('123 Main St');
    await page.getByLabel('City').fill('New York');
    await page.getByRole('button', { name: 'Continue to Payment' }).click();

    // Complete payment (mock)
    await page.getByLabel('Card Number').fill('4242424242424242');
    await page.getByRole('button', { name: 'Pay Now' }).click();

    // Verify confirmation
    await expect(page.getByText('Order Confirmed!')).toBeVisible();
  });
});
```

## Cypress E2E Alternative

```javascript
// cypress/e2e/login.cy.js
describe('Login Page', () => {
  beforeEach(() => {
    cy.visit('/login');
  });

  it('successfully logs in', () => {
    cy.intercept('POST', '/api/login', {
      statusCode: 200,
      body: { token: 'fake-token', user: { name: 'John' } }
    }).as('loginRequest');

    cy.get('[data-cy=email]').type('user@test.com');
    cy.get('[data-cy=password]').type('password123');
    cy.get('[data-cy=submit]').click();

    cy.wait('@loginRequest');
    cy.url().should('include', '/dashboard');
    cy.contains('Welcome, John').should('be.visible');
  });

  it('handles API errors', () => {
    cy.intercept('POST', '/api/login', {
      statusCode: 401,
      body: { error: 'Invalid credentials' }
    }).as('failedLogin');

    cy.get('[data-cy=email]').type('wrong@test.com');
    cy.get('[data-cy=password]').type('wrongpass');
    cy.get('[data-cy=submit]').click();

    cy.wait('@failedLogin');
    cy.contains('Invalid credentials').should('be.visible');
  });
});

// cypress/support/commands.js - Custom Commands
Cypress.Commands.add('login', (email, password) => {
  cy.session([email, password], () => {
    cy.visit('/login');
    cy.get('[data-cy=email]').type(email);
    cy.get('[data-cy=password]').type(password);
    cy.get('[data-cy=submit]').click();
    cy.url().should('include', '/dashboard');
  });
});
```

### Cypress vs Playwright

| Feature | Cypress | Playwright |
|---------|---------|------------|
| **Browsers** | Chrome, Firefox, Edge | Chrome, Firefox, Safari, Edge |
| **Parallelization** | Paid (Dashboard) | Free, built-in |
| **Speed** | Fast | Very fast |
| **Multi-tab** | Limited | Full support |
| **Network mocking** | `cy.intercept()` | `page.route()` |
| **Auto-wait** | Built-in | Built-in |
| **Component testing** | Yes | Experimental |

## Visual Regression & Snapshot Testing

### Snapshot Testing (Jest)

```javascript
// Button.test.jsx
import { render } from '@testing-library/react';
import Button from './Button';

describe('Button', () => {
  it('matches snapshot for primary variant', () => {
    const { container } = render(<Button variant="primary">Click me</Button>);
    expect(container.firstChild).toMatchSnapshot();
  });

  it('matches snapshot for disabled state', () => {
    const { container } = render(<Button disabled>Disabled</Button>);
    expect(container.firstChild).toMatchSnapshot();
  });

  // Inline snapshots for simpler cases
  it('renders correct className', () => {
    const { container } = render(<Button size="large">Large</Button>);
    expect(container.firstChild.className).toMatchInlineSnapshot(`"btn btn-large"`);
  });
});
```

### Visual Regression with Playwright

```javascript
// e2e/visual.spec.ts
import { test, expect } from '@playwright/test';

test.describe('Visual Regression', () => {
  test('homepage matches screenshot', async ({ page }) => {
    await page.goto('/');
    await expect(page).toHaveScreenshot('homepage.png', {
      fullPage: true,
      maxDiffPixelRatio: 0.01
    });
  });

  test('button states', async ({ page }) => {
    await page.goto('/components/button');

    const button = page.getByRole('button', { name: 'Submit' });

    // Default state
    await expect(button).toHaveScreenshot('button-default.png');

    // Hover state
    await button.hover();
    await expect(button).toHaveScreenshot('button-hover.png');

    // Focus state
    await button.focus();
    await expect(button).toHaveScreenshot('button-focus.png');
  });

  test('responsive design', async ({ page }) => {
    await page.goto('/dashboard');

    // Desktop
    await page.setViewportSize({ width: 1920, height: 1080 });
    await expect(page).toHaveScreenshot('dashboard-desktop.png');

    // Tablet
    await page.setViewportSize({ width: 768, height: 1024 });
    await expect(page).toHaveScreenshot('dashboard-tablet.png');

    // Mobile
    await page.setViewportSize({ width: 375, height: 667 });
    await expect(page).toHaveScreenshot('dashboard-mobile.png');
  });
});
```

### Storybook + Chromatic

```javascript
// Button.stories.tsx
import type { Meta, StoryObj } from '@storybook/react';
import Button from './Button';

const meta: Meta<typeof Button> = {
  title: 'Components/Button',
  component: Button,
  parameters: {
    chromatic: { viewports: [320, 768, 1200] }
  }
};

export default meta;
type Story = StoryObj<typeof Button>;

export const Primary: Story = {
  args: { variant: 'primary', children: 'Primary Button' }
};

export const AllVariants: Story = {
  render: () => (
    <div style={{ display: 'flex', gap: '1rem' }}>
      <Button variant="primary">Primary</Button>
      <Button variant="secondary">Secondary</Button>
      <Button variant="danger">Danger</Button>
      <Button disabled>Disabled</Button>
    </div>
  )
};
```

## Testing Best Practices Summary

### ✅ Do

| Practice | Reason |
|----------|--------|
| Test behavior, not implementation | Tests survive refactors |
| Use accessible queries (`getByRole`) | Better accessibility, realistic tests |
| Mock at the network level (MSW) | Tests real fetch/axios code |
| Write fewer E2E, more integration | Balance speed and confidence |
| Use `userEvent` over `fireEvent` | Simulates real user interactions |
| Keep tests isolated | No shared state between tests |
| Test error states | Apps fail; handle it gracefully |

### ❌ Don't

| Anti-Pattern | Why |
|--------------|-----|
| Test implementation details | Brittle tests that break on refactor |
| Excessive `data-testid` usage | Ignores accessibility |
| Snapshot everything | Meaningless, noisy diffs |
| Skip async/await | Flaky tests |
| Mock everything | Miss integration bugs |
| Write E2E for edge cases | Too slow, use unit tests |

### Coverage Guidelines

```javascript
// jest.config.js
module.exports = {
  coverageThreshold: {
    global: {
      branches: 80,
      functions: 80,
      lines: 80,
      statements: 80
    }
  }
};
```

### Test File Organization

```
src/
├── components/
│   ├── Button/
│   │   ├── Button.tsx
│   │   ├── Button.test.tsx       # Unit/integration tests
│   │   └── Button.stories.tsx    # Storybook + visual tests
├── hooks/
│   ├── useAuth.ts
│   └── useAuth.test.ts
├── utils/
│   ├── formatters.ts
│   └── formatters.test.ts
e2e/
├── auth.spec.ts                   # E2E tests
├── checkout.spec.ts
└── visual.spec.ts
mocks/
├── handlers.ts                    # MSW handlers
└── server.ts
```